In [1]:
import pandas as pd
import torch
from transformers import pipeline
import time

In [3]:
# Load the data from the Excel file
file_path = "..\\..\\Data\processed\sample\\twcs_structured_UniqueCount-10_time-20241222-1549.xlsx"
sample_data = pd.read_excel(file_path)

# Display the first few rows of the data to understand its structure
sample_data.head()

,user_id,conversations,company_name,cleaned_conversations,structured_conversations
0,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio..."
1,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio..."
2,115712,Customer: @sprintcare is the worst customer se...,sprintcare,Customer is the worst customer service\nCompan...,"[{'Company_name': 'sprintcare'}, {'conversatio..."
3,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[{'Company_name': 'sprintcare'}, {'conversatio..."
4,115712,Customer: @115714 @sprintcare how can I get in...,sprintcare,Customer how can I get in touch with someone t...,"[{'Company_name': 'sprintcare'}, {'conversatio..."


In [4]:
# Load the Llama pipeline
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
system_prompt = """You are a conversational assistant designed to analyze customer support conversations. Your task is to:
1. Extract entities like product, service, issue type, and others relevant to the conversation.
2. Classify the conversation into one or more intents.
3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.
4. 
---

### **Intents**
Here are the possible intents that you MUST classify:
1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.
2. **Request for Assistance**: Requests for help, support, or clarification.
3. **Inquiry**: Questions about products, services, or policies.
4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).
5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.
6. **Account or Billing Issue**: Problems or queries about account management, billing, refunds, or payment.
7. **Technical Support**: Requests for troubleshooting or technical assistance.
8. **Delivery or Fulfillment Issue**: Issues with order delivery, delays, or couriers.
9. **Promotion or Offer Inquiry**: Questions or complaints about promotions or offers.
10. **Uncategorized**: Messages that don't fit into the above categories.

---

### **Entities**
You must extract the following entities wherever applicable:
- **Product**: Name or type of product mentioned in the conversation.
- **Service**: Name or type of service discussed.
- **Issue Type**: Nature of the problem or complaint.
- **Time Frame**: Any time period or duration mentioned.
- **Other**: Any other relevant information.

---

### **Input Structure**
The input is a JSON object with the following format:
[
  {
    "Company_name": "company_name_here"
  },
  {
    "conversation": [
      {
        "role": "Customer",
        "message": "text_here"
      },
      {
        "role": "Company",
        "message": "text_here"
      }
    ]
  }
]

- **Company_name**: The name of the company involved in the conversation.
- **Conversation**: A list of messages exchanged, where:
  - **role**: Indicates who is speaking (`Customer` or `Company`).
  - **message**: The content of the message.

---

### **Output Structure**
The output MUST be a JSON object with the following format:
{
  "Company_name": "sprintcare",
  "entities": [
    {
      "role": "Customer",
      "entities": {
        "product": "customer service",
        "service": null,
        "issue_type": "poor service",
        "time_frame": null,
        "other": null
      }
    },
    {
      "role": "Company",
      "entities": {
        "product": null,
        "service": "account review",
        "issue_type": null,
        "time_frame": null,
        "other": null
      }
    }
  ],
  "classified_intents": ["Complaint", "Request for Assistance"]
}
### END OF FORMAT
---

### **What You MUST Do**
1. Extract entities from every message in the conversation.
2. Classify intents using the defined categories only.
3. Output both entities and intents in the specified format.
4. Ensure all roles and messages in the input have corresponding entity entries in the output.
5. YOU HAVE TO USE OUTPUT STRUCTURE AND DO NOT ADD ANY EXTRA TEXT BEFORE OR AFTER THE OUTPUT STRUCTURE.
6. JUST ANSWER WITH OUTPUT STRUCTURE.
7. You MUST select at least one intent.
8. "Company_name", "classified_intents" and "entities" fields in the output structure MUST be filled.
9. There MUST be only one customer and company in "entities" structure
---

### **What You MUST NOT Do**
1. Do not invent new entity or intent categories.
2. Do not output intents or entities not listed in the specifications.
3. Do not omit any part of the input conversation from the output.
4. THERE MUST BE ONE CUSTOMER AND ONE COMPANY IN THE OUTPUT STRUCTURE.
---

"""

In [15]:
# Define a function for entity extraction
def extract_entities(conversation_text):
    """
    Extract intents or relevant entities from structured conversation text.
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Identify all relevant entities in this conversation: '{conversation_text}'"},
    ]
    output = pipe(messages, max_new_tokens=256)

    # Extract and return the generated content
    return output[0]["generated_text"][-1].get("content", "") if output else ""

In [16]:
# Add intents to the dataset
def add_intents_to_data(data, text_column="structured_conversations", intent_column="intents", subset=None):
    """
    Add a new column with extracted intents to the dataset.
    Optionally process only a subset of the data.
    """
    # If subset is provided, slice the data
    if subset:
        data = data.iloc[subset]
    
    # Extract intents for each conversation
    data[intent_column] = data[text_column].apply(lambda text: extract_entities(text))
    
    return data

In [17]:
# Define subset range (Optional)
subset_range = None  # Example: Only process rows 1441 to 1442

# Add intents to the dataset
updated_data = add_intents_to_data(sample_data, subset=subset_range) if subset_range else add_intents_to_data(sample_data)

# Save the updated dataset to an Excel file
unique_user_count = len(sample_data["user_id"].unique())
output_file_path = f'..\\..\\Data\\processed\\Intents\\ExtractedIntents_UniqueCount-{unique_user_count}_time-{time.strftime("%Y%m%d-%H%M")}.xlsx'
updated_data.to_excel(output_file_path, index=False)

print(f"Updated dataset with intents saved to {output_file_path}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'system', 'content': 'You are a conversational assistant designed to analyze customer support conversations. Your task is to:\n1. Extract entities like product, service, issue type, and others relevant to the conversation.\n2. Classify the conversation into one or more intents.\n3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.\n4. \n---\n\n### **Intents**\nHere are the possible intents that you MUST classify:\n1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.\n2. **Request for Assistance**: Requests for help, support, or clarification.\n3. **Inquiry**: Questions about products, services, or policies.\n4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).\n5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.\n6. **Account or Billing Issue**: Problems or quer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'system', 'content': 'You are a conversational assistant designed to analyze customer support conversations. Your task is to:\n1. Extract entities like product, service, issue type, and others relevant to the conversation.\n2. Classify the conversation into one or more intents.\n3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.\n4. \n---\n\n### **Intents**\nHere are the possible intents that you MUST classify:\n1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.\n2. **Request for Assistance**: Requests for help, support, or clarification.\n3. **Inquiry**: Questions about products, services, or policies.\n4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).\n5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.\n6. **Account or Billing Issue**: Problems or quer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

[{'generated_text': [{'role': 'system', 'content': 'You are a conversational assistant designed to analyze customer support conversations. Your task is to:\n1. Extract entities like product, service, issue type, and others relevant to the conversation.\n2. Classify the conversation into one or more intents.\n3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.\n4. \n---\n\n### **Intents**\nHere are the possible intents that you MUST classify:\n1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.\n2. **Request for Assistance**: Requests for help, support, or clarification.\n3. **Inquiry**: Questions about products, services, or policies.\n4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).\n5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.\n6. **Account or Billing Issue**: Problems or quer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'system', 'content': 'You are a conversational assistant designed to analyze customer support conversations. Your task is to:\n1. Extract entities like product, service, issue type, and others relevant to the conversation.\n2. Classify the conversation into one or more intents.\n3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.\n4. \n---\n\n### **Intents**\nHere are the possible intents that you MUST classify:\n1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.\n2. **Request for Assistance**: Requests for help, support, or clarification.\n3. **Inquiry**: Questions about products, services, or policies.\n4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).\n5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.\n6. **Account or Billing Issue**: Problems or quer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'system', 'content': 'You are a conversational assistant designed to analyze customer support conversations. Your task is to:\n1. Extract entities like product, service, issue type, and others relevant to the conversation.\n2. Classify the conversation into one or more intents.\n3. In all conversations there is only one customer and one company. DO NOT consider multiple customers or companies.\n4. \n---\n\n### **Intents**\nHere are the possible intents that you MUST classify:\n1. **Complaint**: Expressions of dissatisfaction or frustration with a product, service, or experience.\n2. **Request for Assistance**: Requests for help, support, or clarification.\n3. **Inquiry**: Questions about products, services, or policies.\n4. **Feedback**: Opinions, suggestions, or gratitude (positive, negative, or neutral).\n5. **Escalation Request**: Requests to speak to a manager or higher authority for faster resolution.\n6. **Account or Billing Issue**: Problems or quer